<a href="https://colab.research.google.com/github/elli-saberi/Amazon-reviews-sentiment-analysis/blob/eda/6_Top_Brands_by_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Identify the top 10 brands with the most reviews, then calculate and display the average rating for each brand, sorted by average rating.

set up cell

In [1]:
import pandas as pd


Data Loading and Exploration




In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_path = '/content/drive/My Drive/sentiment analysis-ph2p1/train_data.csv'
brand_path = '/content/drive/My Drive/sentiment analysis-ph2p1/title_brand.csv'

train_df = pd.read_csv(train_path)
brand_df = pd.read_csv(brand_path)

/tmp/ipython-input-1605527540.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_path)


Get a sneak peek of data

In [4]:
train_df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,2,NaN,False,2016-11-11,A2OSUEZJIN7BI,0511189877,NaN,Chris,I have an older URC-WR7 remote and thought thi...,Cannot Learn,1478822400
1,5,NaN,True,2016-06-06,A2NETQRG6JHIG7,0511189877,NaN,Qrysta White,First time I've EVER had a remote that needed ...,zero programming needed! Miracle!?,1465171200
2,4,NaN,True,2016-03-10,A12JHGROAX49G7,0511189877,NaN,Linwood,Got them and only 2 of them worked. company ca...,Works Good and programs easy.,1457568000
3,5,NaN,True,2016-01-14,A1KV65E2TMMG6F,0511189877,NaN,Dane Williams,I got tired of the remote being on the wrong s...,Same as TWC remote,1452729600
4,5,NaN,True,2016-10-20,A280POPEWI0NSA,0594459451,NaN,Kristina H.,After purchasing cheap cords from another webs...,Good Quality Cord,1476921600


In [5]:

brand_df.head()

,asin,title,brand
0,0011300000,Genuine Geovision 1 Channel 3rd Party NVR IP S...,GeoVision
1,0043396828,"Books ""Handbook of Astronomical Image Processi...",33 Books Co.
2,0060009810,One Hot Summer,Visit Amazon's Carolina Garcia Aguilera Page
3,0060219602,Hurray for Hattie Rabbit: Story and pictures (...,Visit Amazon's Dick Gackenbach Page
4,0060786817,sex.lies.murder.fame.: A Novel,Visit Amazon's Lolita Files Page


Let's have a look on the columns and their data types using detailed info function

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838944 entries, 0 to 838943
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   overall         838944 non-null  int64 
 1   vote            191468 non-null  object
 2   verified        838944 non-null  bool  
 3   reviewTime      838944 non-null  object
 4   reviewerID      838944 non-null  object
 5   asin            838944 non-null  object
 6   style           490613 non-null  object
 7   reviewerName    838717 non-null  object
 8   reviewText      838944 non-null  object
 9   summary         838868 non-null  object
 10  unixReviewTime  838944 non-null  int64 
dtypes: bool(1), int64(2), object(8)
memory usage: 64.8+ MB


In [7]:
train_df.isnull().sum()

,0
overall,0
vote,647476
verified,0
reviewTime,0
reviewerID,0
asin,0
style,348331
reviewerName,227
reviewText,0
summary,76


In [8]:
brand_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786445 entries, 0 to 786444
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   asin    786445 non-null  object
 1   title   786426 non-null  object
 2   brand   780947 non-null  object
dtypes: object(3)
memory usage: 18.0+ MB


In [9]:
brand_df.isnull().sum()

,0
asin,0
title,19
brand,5498


 Merge the training data with the brand data

In [10]:
# Merge the two DataFrames based on the common 'asin' column.
# We use a 'left' merge to ensure that all original reviews from train_df are kept,
# and brand information is added where a match is found.
merged_df = pd.merge(train_df, brand_df, on='asin', how='left')

print("--- After Merging (first 5 rows) ---")
print(merged_df.head())

print("\nChecking for null values in 'brand' column after merge...")
print(f"Number of nulls in brand: {merged_df['brand'].isnull().sum()}")

--- After Merging (first 5 rows) ---
   overall vote  verified  reviewTime      reviewerID        asin style  \
0        2  NaN     False  2016-11-11   A2OSUEZJIN7BI  0511189877   NaN   
1        5  NaN      True  2016-06-06  A2NETQRG6JHIG7  0511189877   NaN   
2        4  NaN      True  2016-03-10  A12JHGROAX49G7  0511189877   NaN   
3        5  NaN      True  2016-01-14  A1KV65E2TMMG6F  0511189877   NaN   
4        5  NaN      True  2016-10-20  A280POPEWI0NSA  0594459451   NaN   

    reviewerName                                         reviewText  \
0          Chris  I have an older URC-WR7 remote and thought thi...   
1   Qrysta White  First time I've EVER had a remote that needed ...   
2        Linwood  Got them and only 2 of them worked. company ca...   
3  Dane Williams  I got tired of the remote being on the wrong s...   
4    Kristina H.  After purchasing cheap cords from another webs...   

                               summary  unixReviewTime  \
0                         C

Find the top 10 brands by the total number of reviews

In [12]:
top_10_brands_by_count = merged_df['brand'].value_counts().head(10)

top_brand_names = top_10_brands_by_count.index.tolist()

print("\nTop 10 Brands by Number of Reviews")
print(top_10_brands_by_count)


Top 10 Brands by Number of Reviews
brand
Logitech        17066
Samsung         15409
AmazonBasics    13213
Sony            12436
Asus            11928
Anker            9860
TP-LINK          9773
SanDisk          8400
Sabrent          7320
NETGEAR          7085
Name: count, dtype: int64


Calculate average scores for these top brands and display

In [15]:
# First, filter the merged_df to only contain rows for our top 10 brands.
top_brands_df = merged_df[merged_df['brand'].isin(top_brand_names)]

# Now, group this filtered DataFrame by brand, calculate the mean of the 'overall' score,
# sort the results in descending order.
brand_avg_scores = (
    top_brands_df.groupby('brand')['overall']
    .mean()
    .sort_values(ascending=False)
    .reset_index(name='average_score')
)

print("\n Average Score for Top 10 Brands (Sorted by Score)")
brand_avg_scores


 Average Score for Top 10 Brands (Sorted by Score)


,brand,average_score
0,Anker,4.244320
1,AmazonBasics,4.201468
2,Sabrent,4.028005
3,SanDisk,3.958810
4,Logitech,3.954061
5,Samsung,3.941268
6,Sony,3.938726
7,TP-LINK,3.928681
8,Asus,3.855550
9,NETGEAR,3.836274
